# Flights EDA

In [96]:
import pandas as pd
import numpy as np
from scipy import stats

I spoke to Socorro and she told me that I would have to 
figure out a way to merge the features I want from the respective tables to do the modelling. This will have to be done using 
Join obviously.

I will also have to do the same to the **flights_test** table to ensure I have the same features I trained on. 
I can also delete the features that I won't be using from the **flights_test** table.

Good link https://mode.com/python-tutorial/pandas-groupby-and-python-lambda-functions/

In [97]:
flights_df_testing = pd.read_csv('data/flights_df.csv') # A copy of the original SQL pull is done so that I don't have to requery the db if I shit up

In [98]:
# settings to display all columns
pd.set_option("display.max_columns", None)

In [99]:
flights_df_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146485 entries, 0 to 146484
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   fl_date              146485 non-null  object 
 1   mkt_unique_carrier   146485 non-null  object 
 2   branded_code_share   146485 non-null  object 
 3   mkt_carrier          146485 non-null  object 
 4   mkt_carrier_fl_num   146485 non-null  int64  
 5   op_unique_carrier    146485 non-null  object 
 6   tail_num             146357 non-null  object 
 7   op_carrier_fl_num    146485 non-null  int64  
 8   origin_airport_id    146485 non-null  int64  
 9   origin               146485 non-null  object 
 10  origin_city_name     146485 non-null  object 
 11  dest_airport_id      146485 non-null  int64  
 12  dest                 146485 non-null  object 
 13  dest_city_name       146485 non-null  object 
 14  crs_dep_time         146485 non-null  int64  
 15  dep_time         

In [100]:
flights_df_testing.shape

(146485, 42)

#### Checking for null values and deleting the instances where there is missing values for the target variable

In [101]:
flights_df_testing.isnull().sum()

fl_date                     0
mkt_unique_carrier          0
branded_code_share          0
mkt_carrier                 0
mkt_carrier_fl_num          0
op_unique_carrier           0
tail_num                  128
op_carrier_fl_num           0
origin_airport_id           0
origin                      0
origin_city_name            0
dest_airport_id             0
dest                        0
dest_city_name              0
crs_dep_time                0
dep_time                 1077
dep_delay                1077
taxi_out                 1106
wheels_off               1106
wheels_on                1182
taxi_in                  1182
crs_arr_time                0
arr_time                 1182
arr_delay                1432
cancelled                   0
cancellation_code      145364
diverted                    0
dup                         0
crs_elapsed_time            0
actual_elapsed_time      1432
air_time                 1432
flights                     0
distance                    0
carrier_de

In [102]:
flights_df_testing.dropna(axis=0, how='any', subset=['arr_delay'], inplace=True)

In [103]:
flights_df_testing.isnull().sum()

fl_date                     0
mkt_unique_carrier          0
branded_code_share          0
mkt_carrier                 0
mkt_carrier_fl_num          0
op_unique_carrier           0
tail_num                    0
op_carrier_fl_num           0
origin_airport_id           0
origin                      0
origin_city_name            0
dest_airport_id             0
dest                        0
dest_city_name              0
crs_dep_time                0
dep_time                    0
dep_delay                   0
taxi_out                    0
wheels_off                  0
wheels_on                   0
taxi_in                     0
crs_arr_time                0
arr_time                    0
arr_delay                   0
cancelled                   0
cancellation_code      145053
diverted                    0
dup                         0
crs_elapsed_time            0
actual_elapsed_time         0
air_time                    0
flights                     0
distance                    0
carrier_de

In [104]:
flights_df_testing['op_unique_carrier'].value_counts()
# pd.DataFrame(flights_df_testing['op_unique_carrier'].unique()).value_counts()

WN    25472
AA    17794
DL    15278
OO    13995
UA    11526
MQ     5871
B6     5795
YX     5508
OH     5368
AS     4840
YV     4204
9E     4192
NK     3720
EV     3003
QX     2338
F9     2292
PT     2190
ZW     2081
G4     2005
CP     1794
HA     1622
G7     1564
AX     1381
C5     1004
EM      162
KS       54
Name: op_unique_carrier, dtype: int64

In [105]:
flights_df_testing.head(3)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-01-01,AA,AA_CODESHARE,AA,3069,OO,N727SK,3069,14107,PHX,"Phoenix, AZ",15376,TUS,"Tucson, AZ",1625,1622.0,-3.0,26.0,1648.0,1714.0,3.0,1714,1717.0,3.0,0.0,NaN,0.0,N,49.0,55.0,26.0,1.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,AA,AA_CODESHARE,AA,5606,OH,N575NN,5606,13577,MYR,"Myrtle Beach, SC",11057,CLT,"Charlotte, NC",2157,2158.0,1.0,10.0,2208.0,2248.0,20.0,2311,2308.0,-3.0,0.0,NaN,0.0,N,74.0,70.0,40.0,1.0,157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,10529,BDL,"Hartford, CT",11278,DCA,"Washington, DC",1151,1212.0,21.0,7.0,1219.0,1328.0,3.0,1325,1331.0,6.0,0.0,NaN,0.0,N,94.0,79.0,69.0,1.0,313.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Converting features into applicable types 

In [106]:
flights_df_testing['mkt_carrier_fl_num'] = flights_df_testing['mkt_carrier_fl_num'].astype('category')
flights_df_testing['op_carrier_fl_num'] = flights_df_testing['op_carrier_fl_num'].astype('category')
flights_df_testing['origin_airport_id'] = flights_df_testing['origin_airport_id'].astype('category')
flights_df_testing['dest_airport_id'] = flights_df_testing['dest_airport_id'].astype('category')

# convert date_time column to datetime type
flights_df_testing['fl_date'] = pd.to_datetime(flights_df_testing['fl_date'])


In [108]:
flights_df_testing.head(3)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-01-01,AA,AA_CODESHARE,AA,3069,OO,N727SK,3069,14107,PHX,"Phoenix, AZ",15376,TUS,"Tucson, AZ",1625,1622.0,-3.0,26.0,1648.0,1714.0,3.0,1714,1717.0,3.0,0.0,NaN,0.0,N,49.0,55.0,26.0,1.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,AA,AA_CODESHARE,AA,5606,OH,N575NN,5606,13577,MYR,"Myrtle Beach, SC",11057,CLT,"Charlotte, NC",2157,2158.0,1.0,10.0,2208.0,2248.0,20.0,2311,2308.0,-3.0,0.0,NaN,0.0,N,74.0,70.0,40.0,1.0,157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,10529,BDL,"Hartford, CT",11278,DCA,"Washington, DC",1151,1212.0,21.0,7.0,1219.0,1328.0,3.0,1325,1331.0,6.0,0.0,NaN,0.0,N,94.0,79.0,69.0,1.0,313.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
flights_df_testing.dtypes

fl_date                datetime64[ns]
mkt_unique_carrier             object
branded_code_share             object
mkt_carrier                    object
mkt_carrier_fl_num           category
op_unique_carrier              object
tail_num                       object
op_carrier_fl_num            category
origin_airport_id            category
origin                         object
origin_city_name               object
dest_airport_id              category
dest                           object
dest_city_name                 object
crs_dep_time                    int64
dep_time                      float64
dep_delay                     float64
taxi_out                      float64
wheels_off                    float64
wheels_on                     float64
taxi_in                       float64
crs_arr_time                    int64
arr_time                      float64
arr_delay                     float64
cancelled                     float64
cancellation_code              object
diverted    

In [74]:
# Create a Unique Column to serve as a key when joining to the .groupby() Passengers table
# Key = 'op_unique_carrier' + 'origin_airport_id' + 'dest_airport_id'
flights_df_testing['Carrier_Origin_Dest_Airport'] = flights_df_testing['op_unique_carrier'].astype('str') + flights_df_testing['origin_airport_id'].astype('str') + flights_df_testing['dest_airport_id'].astype('str')

In [75]:
flights_df_testing.head(3)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,Carrier_Origin_Dest_Airport
0,2019-01-01,AA,AA_CODESHARE,AA,3069,OO,N727SK,3069,14107,PHX,"Phoenix, AZ",15376,TUS,"Tucson, AZ",1625,1622.0,-3.0,26.0,1648.0,1714.0,3.0,1714,1717.0,3.0,0.0,NaN,0.0,N,49.0,55.0,26.0,1.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OO1410715376
1,2019-01-01,AA,AA_CODESHARE,AA,5606,OH,N575NN,5606,13577,MYR,"Myrtle Beach, SC",11057,CLT,"Charlotte, NC",2157,2158.0,1.0,10.0,2208.0,2248.0,20.0,2311,2308.0,-3.0,0.0,NaN,0.0,N,74.0,70.0,40.0,1.0,157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OH1357711057
2,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,10529,BDL,"Hartford, CT",11278,DCA,"Washington, DC",1151,1212.0,21.0,7.0,1219.0,1328.0,3.0,1325,1331.0,6.0,0.0,NaN,0.0,N,94.0,79.0,69.0,1.0,313.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OH1052911278


In [76]:
flights_df_testing['Carrier_Origin_Dest_Airport'][0]

'OO1410715376'

### Have to decide on which columns to keep from the flights table

In [11]:
keep_list = ['fl_date', 'mkt_unique_carrier', 'mkt_carrier_fl_num', 'origin_airport_id', 'dest_airport_id', 
             'crs_dep_time','crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance',
             
            'arr_delay']

### Have to decide on which columns to keep and add to the flights_test table

In [12]:
To drop from Table **flights_test**:
['branded_code_share', 'mkt_carrier', 'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin', 
 'origin_city_name', 'dest', 'dest_city_name',  ]

SyntaxError: invalid syntax (<ipython-input-12-d727b18ef962>, line 1)

In [ ]:
To append to Table **flights_test**:


### Feature Engineering Notes

In [ ]:
Feature Engineering Notes
Do something with the 'fl_date'- can convert to hour of the day and day, and also holiday dummy
Combine 'mkt_unique_carrier' and 'mkt_carrier_fl_num' into one feature and dummy
Dummy 'origin_airport_id'
Dummy 'dest_airport_id'
Do something with 'crs_dep_time' and 'crs_arr_time' like bin it into parts of the of the day, quaters, hours. Will have to change type first
Dummy 'dup'


I should include tail number to get some attributes of the aircraft being used like the number of passengers it could hold
vs how many passengers there are, speed of aircraft, 
passengers/seat ratio
average bussiness on runway at each airport 
departure weather
destination weather

Include a feature where the operator is different from the Marketed carrier
Include a feature: Day of the week as a dummy and a weekend flag

For table Passengers combine 'unique_carrier' and 'origin_airport_id' and 'dest_airport_id'
Do the same for Flights table 
then Join on combined key for average seats, passengers, passengers/seats







# Passengers EDA

In [77]:
passengers_df_testing = pd.read_csv('data/passengers_df.csv')

In [78]:
passengers_df_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40479 entries, 0 to 40478
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   departures_scheduled   40479 non-null  float64
 1   departures_performed   40479 non-null  float64
 2   payload                40479 non-null  float64
 3   seats                  40479 non-null  float64
 4   passengers             40479 non-null  float64
 5   freight                40479 non-null  float64
 6   mail                   40479 non-null  float64
 7   distance               40479 non-null  float64
 8   ramp_to_ramp           40479 non-null  float64
 9   air_time               40479 non-null  float64
 10  unique_carrier         40479 non-null  object 
 11  airline_id             40479 non-null  int64  
 12  unique_carrier_name    40479 non-null  object 
 13  region                 40479 non-null  object 
 14  carrier                40479 non-null  object 
 15  ca

In [79]:
passengers_df_testing.isnull().sum()

departures_scheduled     0
departures_performed     0
payload                  0
seats                    0
passengers               0
freight                  0
mail                     0
distance                 0
ramp_to_ramp             0
air_time                 0
unique_carrier           0
airline_id               0
unique_carrier_name      0
region                   0
carrier                  0
carrier_name             0
carrier_group            0
carrier_group_new        0
origin_airport_id        0
origin_city_market_id    0
origin                   0
origin_city_name         0
origin_country           0
origin_country_name      0
dest_airport_id          0
dest_city_market_id      0
dest                     0
dest_city_name           0
dest_country             0
dest_country_name        0
aircraft_group           0
aircraft_type            0
aircraft_config          0
year                     0
month                    0
distance_group           0
class                    0
d

In [80]:
# Create a Unique Column to serve as a key when grouping by so that the resultant grouped data can be merged onto the flights data table
# Key = 'mkt_unique_carrier' + 'origin_airport_id' + 'dest_airport_id'
passengers_df_testing['Carrier_Origin_Dest_Airport'] = passengers_df_testing['unique_carrier'].astype('str') + passengers_df_testing['origin_airport_id'].astype('str') + passengers_df_testing['dest_airport_id'].astype('str')

In [81]:
len(passengers_df_testing['Carrier_Origin_Dest_Airport'].unique())

25231

In [82]:
passengers_df_testing.head(3)

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,unique_carrier,airline_id,unique_carrier_name,region,carrier,carrier_name,carrier_group,carrier_group_new,origin_airport_id,origin_city_market_id,origin,origin_city_name,origin_country,origin_country_name,dest_airport_id,dest_city_market_id,dest,dest_city_name,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source,Carrier_Origin_Dest_Airport
0,0.0,4.0,124800.0,624.0,488.0,0.0,0.0,1396.0,942.0,768.0,G4,20368,Allegiant Air,D,G4,Allegiant Air,3,3,14122,30198,PIT,"Pittsburgh, PA",US,United States,11032,31032,CUN,"Cancun, Mexico",MX,Mexico,6,698,1,2019,1,3,L,IU,G41412211032
1,0.0,1.0,18800.0,70.0,0.0,0.0,0.0,83.0,43.0,18.0,G7,20500,GoJet Airlines LLC d/b/a United Express,D,G7,GoJet Airlines LLC d/b/a United Express,2,2,11057,31057,CLT,"Charlotte, NC",US,United States,11995,31995,GSO,"Greensboro/High Point, NC",US,United States,6,631,1,2019,1,1,F,DU,G71105711995
2,0.0,1.0,22750.0,76.0,17.0,0.0,0.0,308.0,75.0,55.0,G7,20500,GoJet Airlines LLC d/b/a United Express,D,G7,GoJet Airlines LLC d/b/a United Express,2,2,11193,33105,CVG,"Cincinnati, OH",US,United States,15016,31123,STL,"St. Louis, MO",US,United States,6,638,1,2019,1,1,F,DU,G71119315016


In [83]:
# grouping the passengers table by the unique key just created to get the sum of the seats and sum of the passengers per unique key
# this will be saved as a separate df which will be joined onto the flights df
# after which a new feature will be caluclated: Passengers/Seats to give the ratio of passengers to seats to see if this has an effect on the regression.

passengers_df_testing_grouped = passengers_df_testing.groupby('Carrier_Origin_Dest_Airport').agg({'seats': np.sum, 'passengers': np.sum })

In [84]:
passengers_df_testing_grouped.head()

,seats,passengers
Carrier_Origin_Dest_Airport,,
04Q1015412197,16.0,9.0
04Q1025713541,8.0,1.0
04Q1031214843,8.0,2.0
04Q1041410927,8.0,2.0
04Q1044414843,144.0,97.0


In [113]:
passengers_df_testing_grouped.columns.values

array(['seats', 'passengers'], dtype=object)

In [85]:
len(passengers_df_testing_grouped.index)

25231

# Modeling Data

In [86]:
X = flights_df_testing.copy()
print(X.shape)

(145053, 43)


In [87]:
X = X.join(passengers_df_testing_grouped, on = 'Carrier_Origin_Dest_Airport', how = 'left')
print(X.shape)

(145053, 45)


## Feature Engineering

### Feature 1: Passengers_Seat_Ratio

This gives an indication of the demand for the flight

In [90]:
X['Passengers_Seat_Ratio'] =  X['passengers'] / X['seats']

In [91]:
X.head(3)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,Carrier_Origin_Dest_Airport,seats,passengers,Passengers_Seat_Ratio
0,2019-01-01,AA,AA_CODESHARE,AA,3069,OO,N727SK,3069,14107,PHX,"Phoenix, AZ",15376,TUS,"Tucson, AZ",1625,1622.0,-3.0,26.0,1648.0,1714.0,3.0,1714,1717.0,3.0,0.0,NaN,0.0,N,49.0,55.0,26.0,1.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OO1410715376,3950.0,3184.0,0.806076
1,2019-01-01,AA,AA_CODESHARE,AA,5606,OH,N575NN,5606,13577,MYR,"Myrtle Beach, SC",11057,CLT,"Charlotte, NC",2157,2158.0,1.0,10.0,2208.0,2248.0,20.0,2311,2308.0,-3.0,0.0,NaN,0.0,N,74.0,70.0,40.0,1.0,157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OH1357711057,15504.0,12631.0,0.814693
2,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,10529,BDL,"Hartford, CT",11278,DCA,"Washington, DC",1151,1212.0,21.0,7.0,1219.0,1328.0,3.0,1325,1331.0,6.0,0.0,NaN,0.0,N,94.0,79.0,69.0,1.0,313.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OH1052911278,6890.0,5501.0,0.798403


In [114]:
X.seats.isna().sum()

0

### Feature 2: mkt_op_carrier_difference

This gives an indication where a third party would have to service the route on behalf of the Marketed Carrier

In [92]:
X['mkt_op_carrier_difference'] = np.where(X["mkt_unique_carrier"] == X["op_unique_carrier"], True, False)

# X['mkt_op_carrier_difference'] = X['mkt_unique_carrier'].apply(lambda x: x ==  1)
print(X.shape)

(145053, 47)


In [94]:
X.sample(5)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,Carrier_Origin_Dest_Airport,seats,passengers,Passengers_Seat_Ratio,mkt_op_carrier_difference
90883,2019-01-05,HA,HA,HA,101,HA,N489HA,101,12402,ITO,"Hilo, HI",12173,HNL,"Honolulu, HI",619,611.0,-8.0,7.0,618.0,654.0,9.0,710,703.0,-7.0,0.0,NaN,0.0,N,51.0,52.0,36.0,1.0,216.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HA1240212173,51072.0,41198.0,0.806665,True
65482,2019-01-04,WN,WN,WN,1890,WN,N461WN,1890,10397,ATL,"Atlanta, GA",14524,RIC,"Richmond, VA",2155,2201.0,6.0,14.0,2215.0,2316.0,3.0,2325,2319.0,-6.0,0.0,NaN,0.0,N,90.0,78.0,61.0,1.0,481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WN1039714524,12394.0,8663.0,0.698967,True
129707,2019-01-07,WN,WN,WN,3338,WN,N291WN,3338,10423,AUS,"Austin, TX",14107,PHX,"Phoenix, AZ",655,654.0,-1.0,12.0,706.0,822.0,6.0,830,828.0,-2.0,0.0,NaN,0.0,N,155.0,154.0,136.0,1.0,872.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WN1042314107,19364.0,15337.0,0.792037,True
53086,2019-01-03,AA,AA_CODESHARE,AA,3992,MQ,N247NN,3992,11193,CVG,"Cincinnati, OH",11298,DFW,"Dallas/Fort Worth, TX",1942,2006.0,24.0,10.0,2016.0,2133.0,14.0,2129,2147.0,18.0,0.0,NaN,0.0,N,167.0,161.0,137.0,1.0,812.0,0.0,0.0,0.0,0.0,18.0,NaN,NaN,NaN,NaN,MQ1119311298,3844.0,3258.0,0.847555,False
17676,2019-01-01,UA,UA_CODESHARE,UA,3431,YX,N862RW,3431,11618,EWR,"Newark, NJ",14492,RDU,"Raleigh/Durham, NC",1100,1055.0,-5.0,22.0,1117.0,1234.0,7.0,1253,1241.0,-12.0,0.0,NaN,0.0,N,113.0,106.0,77.0,1.0,416.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YX1161814492,10274.0,8466.0,0.824022,False


### Feature 3: Most common aircraft_group and class used 

https://stackoverflow.com/questions/15222754/groupby-pandas-dataframe-and-select-most-common-value

In [25]:
# passengers_df_testing_grouped2 = pd.DataFrame(passengers_df_testing.groupby('Carrier_Origin_Dest_Airport')['aircraft_group'].agg(pd.Series.mode))
passengers_df_testing.groupby(['Carrier_Origin_Dest_Airport'])['origin_airport_id'].agg(pd.Series.mode).to_frame()

# I dont know why 'aircraft_type' and 'aircraft_group' is not working!!                                       

,origin_airport_id
Carrier_Origin_Dest_Airport,
04Q1015412197,10154
04Q1025713541,10257
04Q1031214843,10312
04Q1041410927,10414
04Q1044414843,10444
...,...
ZX1627113342,16271
ZX1627113871,16271
ZX1627114122,16271


In [26]:
# passengers_df_testing.groupby(['Carrier_Origin_Dest_Airport'])['class'].agg(pd.Series.mode).to_frame() aircraft_group

passengers_df_testing.groupby(['Carrier_Origin_Dest_Airport'])['class'].agg(pd.Series.mode).to_frame()

,class
Carrier_Origin_Dest_Airport,
04Q1015412197,L
04Q1025713541,L
04Q1031214843,L
04Q1041410927,L
04Q1044414843,"[F, L]"
...,...
ZX1627113342,F
ZX1627113871,F
ZX1627114122,F


In [27]:
# I thought I could try tricking it by just merging or joining in the hopes it would take the first value but it is not

In [28]:
# X = X.merge(passengers_df_testing[['Carrier_Origin_Dest_Airport', "aircraft_group", "distance_group", 'class']], how='inner', on=['Carrier_Origin_Dest_Airport'])
# print(X.shape)

In [95]:
# X = X.join(passengers_df_testing[['Carrier_Origin_Dest_Airport', "aircraft_group", "distance_group", 'class']].set_index('Carrier_Origin_Dest_Airport'), on='Carrier_Origin_Dest_Airport', how='left')
# print(X.shape)

### Date Time Features